In [114]:
import pandas as pd
import numpy as np
from statsmodels.stats.inter_rater import fleiss_kappa
import simpledorff

In [115]:
file_name = "../annotated_multipico_new.xlsx"
sheet_names = ['Alessandro', 'Cristina', 'Eliana', 'Michele', 'Michael', 'Elisa']
df = pd.read_excel(file_name, sheet_name=sheet_names)
df['Alessandro'].head()

,level,source,reply,post,language,language_variety,reply_id,post_id,majority_vote,Unnamed: 9,rhetorical figure,comment
0,2,twitter,@USER @USER Anche per rompere il ghiaccio,@USER Gianca già che ci sei regalale un paio d...,it,it,15902900000000000000,7961696982295378944,1,NaN,"CONTEXT SHIFT, HYPERBOLE",NaN
1,2,twitter,@USER Nonnino! Già sveglio? ??,@USER Non troppo buongiorno,it,it,4652170000000000000,4824587311292975104,1,NaN,"RHETORICAL QUESTION, HYPERBOLE",NaN
2,1,twitter,@USER il terrone di essere rimpiazzata,il terrore di essere rimpiazzata,it,it,18161900000000000000,7155693771356047360,1,NaN,"CONTEXT SHIFT, OTHER",NaN
3,1,twitter,@USER Tanto per non essere banali,"Elly #Schlein: ""Il lavoro deve tornare a esser...",it,it,5855670000000000000,5929177359346716672,1,NaN,FALSE ASSERTION,NaN
4,1,reddit,Se le toghe rosse avessere mandato un avviso d...,Se solo la società perseguisse l'obiettivo di ...,it,it,10284800000000000000,12099038560420169728,1,NaN,"OXYMORON, ANALOGY",NaN


In [116]:
file_name = "../annotated_multipico_new.xlsx"
sheet_names = ['Alessandro', 'Cristina', 'Eliana', 'Michele', 'Michael', 'Elisa']

# Carico solo il foglio di Cristina
df_cristina = pd.read_excel(file_name, sheet_name='Cristina')
colonna_rf = df_cristina['rhetorical figure']

# Trovo l'indice dell'ultima riga non vuota nella colonna 'rhetorical figure' di Cristina
ultima_riga_non_vuota = colonna_rf.last_valid_index()
print("L'ultima riga annotata da Cristina (per la colonna rhetorical figure) è:", ultima_riga_non_vuota)

# Ora carico tutti gli sheet
df_dict = pd.read_excel(file_name, sheet_name=sheet_names)

# Taglio ogni DataFrame fino all'ultima riga non vuota annotata da Cristina
# ATTENZIONE: last_valid_index() è l'indice dell'ultima riga valida, potrebbe essere un numero qualsiasi,
# bisogna verificare se è un indice numerico o un indice di riga (se è un RangeIndex tipicamente lo è)
# Assumiamo che sia un indice numerico, quindi per prendere le prime righe useremo .iloc[:ultima_riga_non_vuota + 1]

df_dict_truncated = {}
for sheet, df in df_dict.items():
    df_dict_truncated[sheet] = df.iloc[:ultima_riga_non_vuota + 1]

# Ora df_dict_truncated contiene tutti i DataFrame tagliati al numero di righe di Cristina
df = df_dict_truncated

L'ultima riga annotata da Cristina (per la colonna rhetorical figure) è: 200


In [117]:
# DROP colonne indesiderate subito dopo il caricamento
cols_to_drop = ['level', 'source', 'language', 'language_variety', 'majority_vote', 'Unnamed: 9', 'comment']
for name in df:
    df[name] = df[name].drop(columns=[col for col in cols_to_drop if col in df[name].columns])
df['Alessandro'].head()

,reply,post,reply_id,post_id,rhetorical figure
0,@USER @USER Anche per rompere il ghiaccio,@USER Gianca già che ci sei regalale un paio d...,15902900000000000000,7961696982295378944,"CONTEXT SHIFT, HYPERBOLE"
1,@USER Nonnino! Già sveglio? ??,@USER Non troppo buongiorno,4652170000000000000,4824587311292975104,"RHETORICAL QUESTION, HYPERBOLE"
2,@USER il terrone di essere rimpiazzata,il terrore di essere rimpiazzata,18161900000000000000,7155693771356047360,"CONTEXT SHIFT, OTHER"
3,@USER Tanto per non essere banali,"Elly #Schlein: ""Il lavoro deve tornare a esser...",5855670000000000000,5929177359346716672,FALSE ASSERTION
4,Se le toghe rosse avessere mandato un avviso d...,Se solo la società perseguisse l'obiettivo di ...,10284800000000000000,12099038560420169728,"OXYMORON, ANALOGY"


In [118]:
# Per ogni foglio, tieni solo righe con 'rhetorical figure' non-NaN
'''cleaned = {
    name: sheet[sheet['rhetorical figure'].notna()][['post_id', 'reply_id']]
    for name, sheet in df.items()
}

sets_of_pairs = [set(zip(sheet['post_id'], sheet['reply_id'])) for sheet in cleaned.values()]
common_pairs = set.intersection(*sets_of_pairs)
print(f"Coppie (post_id, reply_id) comuni con 'rhetorical figure' non-NaN in tutti i fogli: {len(common_pairs)}")

final_cleaned = {}
for name, sheet in df.items():
    mask = sheet.apply(lambda row: (row['post_id'], row['reply_id']) in common_pairs, axis=1)
    mask &= sheet['rhetorical figure'].notna()
    final_cleaned[name] = sheet[mask].copy()

for sheet_name in sheet_names:
    print(f"Numero di post in {sheet_name}: {len(final_cleaned[sheet_name])}")

final_cleaned['Alessandro'].head()'''
final_cleaned = df
final_cleaned['Michele'].iloc[27:32]

,reply,post,reply_id,post_id,rhetorical figure
27,Se po buccà chi drento?,Siamo sicuri non sia matteo montesi?,10540900000000000000,1004779540710112000,OTHER
28,@USER @USER Tarlo mentale.,@USER @USER Per esiliare dal suo amico sceicc...,6975850000000000000,10742053278056030208,NaN
29,@USER @USER La giustizia e' aliena su questo ...,@USER @USER Sì ma non è Giusto.,15962200000000000000,14374185223286280192,HYPERBOLE
30,Perché avere delle bottiglie di vino a casa qu...,Io non ho mai capito perché si dovrebbero salv...,5036060000000000000,16167343742615099392,RHETORICAL QUESTION
31,@USER I don’t,@USER Non essere così cattivo con lei...❤️❤️❤️,16985200000000000000,4320113204215145984,NaN


In [119]:
len(final_cleaned['Michele'])

201

In [120]:
# Estrai tutte le figure retoriche da tutti i fogli, normalizzate
all_figures_set = set()

def normalize_figures(cell):
    # Gestione NaN
    if pd.isna(cell):
        return []
    figures = [f.strip().upper() for f in str(cell).split(',')]
    # Sostituzione specifica
    figures = ["OXYMORON" if f == "OXYMORON/PARADOX" else f for f in figures]
    figures = ["EUPHEMISM" if f == "EUPHEMINSM" else f for f in figures]
    return figures

# Raccogli tutte le figure da tutti i fogli
for sheet in final_cleaned.values():
    for entry in sheet['rhetorical figure'].dropna():
        all_figures_set.update(normalize_figures(entry))

# Ordina per consistenza
all_figures = sorted(all_figures_set)

# Aggiungi una colonna per ciascuna figura: 1 se presente, 0 altrimenti
for name, sheet in final_cleaned.items():
    for fig in all_figures:
        sheet[fig] = sheet['rhetorical figure'].apply(
            lambda x: 1 if fig in normalize_figures(x) else 0
        )
    final_cleaned[name] = sheet  # Aggiorna

final_cleaned['Michael'].head()

# drop unnamed: 11 col
final_cleaned['Michael'] = final_cleaned['Michael'].drop(columns=[col for col in final_cleaned['Michael'].columns if 'Unnamed' in col])
final_cleaned['Michael'].head()

,post,reply,reply_id,post_id,rhetorical figure,ANALOGY,CONTEXT SHIFT,EUPHEMISM,FALSE ASSERTION,HYPERBOLE,OTHER,OXYMORON,RHETORICAL QUESTION
0,@USER Gianca già che ci sei regalale un paio d...,@USER @USER Anche per rompere il ghiaccio,15902900000000000000,7961696982295378944,EUPHEMISM,0,0,1,0,0,0,0,0
1,@USER Non troppo buongiorno,@USER Nonnino! Già sveglio? ??,4652170000000000000,4824587311292975104,RHETORICAL QUESTION,0,0,0,0,0,0,0,1
2,il terrore di essere rimpiazzata,@USER il terrone di essere rimpiazzata,18161900000000000000,7155693771356047360,OTHER,0,0,0,0,0,1,0,0
3,"Elly #Schlein: ""Il lavoro deve tornare a esser...",@USER Tanto per non essere banali,5855670000000000000,5929177359346716672,EUPHEMISM,0,0,1,0,0,0,0,0
4,Se solo la società perseguisse l'obiettivo di ...,Se le toghe rosse avessere mandato un avviso d...,10284800000000000000,12099038560420169728,OXYMORON/PARADOX,0,0,0,0,0,0,1,0


# Polarization index

## Funzioni

In [121]:
import numpy as np
import pandas as pd
from scipy.stats import chisquare

def chi2_normalized_fixed(counts):
    counts = np.array(counts)
    n = counts.sum()
    k = len(counts)
    if n == 0 or k <= 1:
        return 1.0  # accordo perfetto
    
    expected = np.full(k, n / k)
    chi2_stat, _ = chisquare(f_obs=counts, f_exp=expected)
    a = (chi2_stat / n)
    return max(0.0, min(1.0, a))

def compute_p_index_for_message_fixed(df_message, category, group_map):
    df_message = df_message.copy()
    df_message['group'] = df_message['annotator'].map(group_map)
    
    # Converti i valori binari in categorie: 'yes' o 'no'
    df_message[category] = df_message[category].apply(lambda x: 'yes' if x == 1 else 'no')
    labels = ['yes', 'no']
    
    counts_global = df_message[category].value_counts().reindex(labels, fill_value=0).values
    aG = chi2_normalized_fixed(counts_global)

    groups = df_message['group'].unique()
    k = len(groups)
    
    aGw_list = []
    for g in groups:
        counts_group = df_message[df_message['group'] == g][category].value_counts().reindex(labels, fill_value=0).values
        aGw = chi2_normalized_fixed(counts_group)
        aGw_list.append(aGw)
    
    p_index = (1/k) * sum(aGw * (1 - aG) for aGw in aGw_list)
    return p_index

## Polarization su generazione

### Real

In [122]:
# gruppi per GENERE
annotator_groups = {
    'Alessandro': 'genX',
    'Michele': 'genY',
    'Michael': 'genZ',
    'Cristina': 'genX',
    'Eliana': 'genY',
    'Elisa': 'genZ'
}
# Crea dataframe con annotazioni binarie per ogni figura
all_annotations = []
for annotator, df_ann in final_cleaned.items():
    df_copy = df_ann.copy()
    df_copy['annotator'] = annotator
    all_annotations.append(df_copy[['post_id', 'reply_id', 'annotator'] + all_figures])
    
full_df = pd.concat(all_annotations, ignore_index=True)
full_df.head()

p_index_per_instance = {}

# Calcolo del p-index medio per istanza (post_id, reply_id)
p_index_per_instance = {}
group_map = annotator_groups  # già definito

instances = full_df[['post_id', 'reply_id']].drop_duplicates()

rhetorical_figures = all_figures  # lista delle colonne tipo ['ANALOGY', 'CONTEXT SHIFT', ...]

In [123]:
for _, row in instances.iterrows():
    pid, rid = row['post_id'], row['reply_id']
    instance_df = full_df[(full_df['post_id'] == pid) & (full_df['reply_id'] == rid)]
    
    p_indexes = []
    for fig in rhetorical_figures:
        #print(f"Calcolo P-index per post_id: {pid}, reply_id: {rid}, figura: {fig}")

        df_subset = instance_df[['annotator', fig]].copy()
        #print(df_subset)

        if df_subset[fig].sum() + (df_subset[fig] == 0).sum() < 2:
            print(f"Troppo pochi annotatori per la figura {fig} in post_id: {pid}, reply_id: {rid}")
            continue  # Troppo pochi annotatori per questa figura
        
        p = compute_p_index_for_message_fixed(df_subset.assign(dummy_id=1), fig, group_map)
        #print(f"P-index per post_id: {pid}, reply_id: {rid}, figura: {fig} è {p:.3f}")
        p_indexes.append(p)

    #break

    if p_indexes:
        p_index_per_instance[(pid, rid)] = np.mean(p_indexes)

# Calcolo della media globale dei P-index
mean_p_index_real = np.mean(list(p_index_per_instance.values()))
print(f"Media del P-index per tutti i post: {mean_p_index_real:.3f}")

# Trova istanza con P-index massimo e minimo
max_instance = max(p_index_per_instance.items(), key=lambda x: x[1])
min_instance = min(p_index_per_instance.items(), key=lambda x: x[1])

print("\n--- ISTANZA CON P-INDEX MASSIMO ---")
print(f"post_id: {max_instance[0][0]}, reply_id: {max_instance[0][1]}, P-index: {max_instance[1]:.3f}")
display(full_df[(full_df['post_id'] == max_instance[0][0]) & (full_df['reply_id'] == max_instance[0][1])])

print("\n--- ISTANZA CON P-INDEX MINIMO ---")
print(f"post_id: {min_instance[0][0]}, reply_id: {min_instance[0][1]}, P-index: {min_instance[1]:.3f}")
display(full_df[(full_df['post_id'] == min_instance[0][0]) & (full_df['reply_id'] == min_instance[0][1])])



Media del P-index per tutti i post: 0.191

--- ISTANZA CON P-INDEX MASSIMO ---
post_id: 1894119167367809024, reply_id: 9033280000000000000, P-index: 0.380


,post_id,reply_id,annotator,ANALOGY,CONTEXT SHIFT,EUPHEMISM,FALSE ASSERTION,HYPERBOLE,OTHER,OXYMORON,RHETORICAL QUESTION
19,1894119167367809024,9033280000000000000,Alessandro,1,0,0,0,0,1,0,0
220,1894119167367809024,9033280000000000000,Cristina,1,0,0,0,0,1,0,0
421,1894119167367809024,9033280000000000000,Eliana,0,0,0,0,0,1,0,0
622,1894119167367809024,9033280000000000000,Michele,0,0,0,0,0,1,0,0
823,1894119167367809024,9033280000000000000,Michael,0,1,0,1,0,0,0,0
1024,1894119167367809024,9033280000000000000,Elisa,0,0,0,1,0,0,0,0



--- ISTANZA CON P-INDEX MINIMO ---
post_id: 9408805770120683520, reply_id: 2035190000000000000, P-index: 0.000


,post_id,reply_id,annotator,ANALOGY,CONTEXT SHIFT,EUPHEMISM,FALSE ASSERTION,HYPERBOLE,OTHER,OXYMORON,RHETORICAL QUESTION
88,9408805770120683520,2035190000000000000,Alessandro,0,0,0,0,1,0,0,0
289,9408805770120683520,2035190000000000000,Cristina,0,0,0,0,1,0,0,0
490,9408805770120683520,2035190000000000000,Eliana,0,0,0,0,1,0,0,0
691,9408805770120683520,2035190000000000000,Michele,0,0,0,0,1,0,0,0
892,9408805770120683520,2035190000000000000,Michael,0,0,0,0,1,0,0,0
1093,9408805770120683520,2035190000000000000,Elisa,0,0,0,0,1,0,0,0


In [131]:
p_index_per_instance

{(7961696982295378944, 15902900000000000000): 0.12962962962962965,
 (4824587311292975104, 4652170000000000000): 0.17746913580246915,
 (7155693771356047360, 18161900000000000000): 0.08950617283950618,
 (5929177359346716672, 5855670000000000000): 0.06635802469135803,
 (12099038560420169728, 10284800000000000000): 0.18981481481481483,
 (6878336134883742720, 15991800000000000000): 0.0910493827160494,
 (18184193230015000576, 3263700000000000000): 0.1527777777777778,
 (4625931298966675456, 3210680000000000000): 0.012345679012345678,
 (457436274759233728, 11247500000000000000): 0.10185185185185186,
 (12841824500967149568, 13599000000000000000): 0.10339506172839508,
 (8374295896122236928, 9960700000000000000): 0.2916666666666667,
 (1439242534760017920, 10568700000000000000): 0.1682098765432099,
 (8727623984665907200, 457492000000000000): 0.18055555555555558,
 (7033897647878070272, 4174370000000000000): 0.08950617283950618,
 (14927487631298500608, 3984510000000000000): 0.20679012345679015,
 (13

In [139]:
# get the post and the reply text given the post_id=1894119167367809024 and reply_id=9033280000000000000
post_id = 7462581045673427968
reply_id = 14235400000000000000
post_text = final_cleaned['Alessandro'][final_cleaned['Alessandro']['post_id'] == post_id]['post'].values[0]
reply_text = final_cleaned['Alessandro'][final_cleaned['Alessandro']['reply_id'] == reply_id]['reply'].values[0]
print(f"Post text: {post_text}")
print(f"Reply text: {reply_text}")



Post text: @USER Sarà la prima squadra forte che affrontano....
Reply text: @USER Quale delle due? ?


### su coppie casuali (insieme delle parti)

In [124]:
import itertools
import pandas as pd

annotators = list(final_cleaned.keys())

# Dimensioni gruppi
group1_size = len(annotators) // 3
group2_size = len(annotators) - group1_size

# Tutte le combinazioni di annotatori
all_combinations = list(itertools.combinations(annotators, group1_size))
print(f"Numero totale combinazioni: {len(all_combinations)}")

# Istanza uniche (post_id, reply_id)
instances = full_df[['post_id', 'reply_id']].drop_duplicates()
rhetorical_figures = all_figures  # lista delle figure retoriche

# Calcolo P-index medio per ogni combinazione di gruppi
p_index_means = []

for i, group1 in enumerate(all_combinations, 1):
    group2 = [a for a in annotators if a not in group1]
    group_map = {a: 'group1' if a in group1 else 'group2' for a in annotators}
    
    p_index_per_instance = {}

    for _, row in instances.iterrows():
        pid, rid = row['post_id'], row['reply_id']
        instance_df = full_df[(full_df['post_id'] == pid) & (full_df['reply_id'] == rid)]

        p_indexes = []
        for fig in rhetorical_figures:
            df_subset = instance_df[['annotator', fig]].copy()
            if df_subset[fig].sum() + (df_subset[fig] == 0).sum() < 2:
                continue  # Troppo pochi annotatori
            p = compute_p_index_for_message_fixed(df_subset.assign(dummy_id=1), fig, group_map)
            p_indexes.append(p)

        if p_indexes:
            p_index_per_instance[(pid, rid)] = np.mean(p_indexes)
    
    # Media dei p-index per istanza in questa combinazione
    if p_index_per_instance:
        mean_p_index = np.mean(list(p_index_per_instance.values()))
        p_index_means.append(mean_p_index)

    if i % 10 == 0 or i == len(all_combinations):
        print(f"Calcolate {i}/{len(all_combinations)} combinazioni, ultima media P-index: {mean_p_index:.3f}")

# Statistiche finali
overall_mean_random = np.mean(p_index_means)
print(f"\nMedia del P-index su tutte le permutazioni: {overall_mean_random:.3f}")

Numero totale combinazioni: 15
Calcolate 10/15 combinazioni, ultima media P-index: 0.152
Calcolate 15/15 combinazioni, ultima media P-index: 0.151

Media del P-index su tutte le permutazioni: 0.146


### Delta

In [125]:
def calc_delta(real, random):
    return ((real - random) / random) * 100

print("\nDelta tra P-index reale e medio casuale:")
print(f"  Reale: {mean_p_index_real:.3f}")
print(f"  Casual: {overall_mean_random:.3f}")

print(calc_delta(mean_p_index_real, overall_mean_random))


Delta tra P-index reale e medio casuale:
  Reale: 0.191
  Casual: 0.146
31.129389433723542


## Polarization su genere

### Real

In [126]:
# from scipy.stats import chisquare

# Esempio: gruppi per GENERE (ipotetico, va adattato ai tuoi dati reali)
annotator_groups = {
    'Alessandro': 'male',
    'Michele': 'male',
    'Michael': 'male',
    'Cristina': 'female',
    'Eliana': 'female',
    'Elisa': 'female'
}

# Crea dataframe con annotazioni binarie per ogni figura
all_annotations = []
for annotator, df_ann in final_cleaned.items():
    df_copy = df_ann.copy()
    df_copy['annotator'] = annotator
    all_annotations.append(df_copy[['post_id', 'reply_id', 'annotator'] + all_figures])

full_df = pd.concat(all_annotations, ignore_index=True)
full_df.head()

p_index_per_instance = {}

# Calcolo del p-index medio per istanza (post_id, reply_id)
p_index_per_instance = {}
group_map = annotator_groups  # già definito

instances = full_df[['post_id', 'reply_id']].drop_duplicates()

rhetorical_figures = all_figures  # lista delle colonne tipo ['ANALOGY', 'CONTEXT SHIFT', ...]

for _, row in instances.iterrows():
    pid, rid = row['post_id'], row['reply_id']
    instance_df = full_df[(full_df['post_id'] == pid) & (full_df['reply_id'] == rid)]
    
    p_indexes = []
    for fig in rhetorical_figures:
        df_subset = instance_df[['annotator', fig]].copy()
        if df_subset[fig].sum() + (df_subset[fig] == 0).sum() < 2:
            continue  # Troppo pochi annotatori per questa figura
        
        p = compute_p_index_for_message_fixed(df_subset.assign(dummy_id=1), fig, group_map)
        p_indexes.append(p)
    
    if p_indexes:
        p_index_per_instance[(pid, rid)] = np.mean(p_indexes)

# Calcolo della media globale dei P-index
mean_p_index_real = np.mean(list(p_index_per_instance.values()))
print(f"Media del P-index per tutti i post: {mean_p_index_real:.3f}")

# Trova istanza con P-index massimo e minimo
max_instance = max(p_index_per_instance.items(), key=lambda x: x[1])
min_instance = min(p_index_per_instance.items(), key=lambda x: x[1])

print("\n--- ISTANZA CON P-INDEX MASSIMO ---")
print(f"post_id: {max_instance[0][0]}, reply_id: {max_instance[0][1]}, P-index: {max_instance[1]:.3f}")
display(full_df[(full_df['post_id'] == max_instance[0][0]) & (full_df['reply_id'] == max_instance[0][1])])

print("\n--- ISTANZA CON P-INDEX MINIMO ---")
print(f"post_id: {min_instance[0][0]}, reply_id: {min_instance[0][1]}, P-index: {min_instance[1]:.3f}")
display(full_df[(full_df['post_id'] == min_instance[0][0]) & (full_df['reply_id'] == min_instance[0][1])])

Media del P-index per tutti i post: 0.124

--- ISTANZA CON P-INDEX MASSIMO ---
post_id: 18085984606050760704, reply_id: 3945920000000000000, P-index: 0.290


,post_id,reply_id,annotator,ANALOGY,CONTEXT SHIFT,EUPHEMISM,FALSE ASSERTION,HYPERBOLE,OTHER,OXYMORON,RHETORICAL QUESTION
183,18085984606050760704,3945920000000000000,Alessandro,0,0,0,0,1,1,0,1
384,18085984606050760704,3945920000000000000,Cristina,0,1,0,0,0,0,1,0
585,18085984606050760704,3945920000000000000,Eliana,1,0,0,1,0,0,0,0
786,18085984606050760704,3945920000000000000,Michele,1,0,0,0,1,0,0,0
987,18085984606050760704,3945920000000000000,Michael,1,0,0,0,1,0,0,0
1188,18085984606050760704,3945920000000000000,Elisa,1,1,0,0,1,0,0,0



--- ISTANZA CON P-INDEX MINIMO ---
post_id: 9408805770120683520, reply_id: 2035190000000000000, P-index: 0.000


,post_id,reply_id,annotator,ANALOGY,CONTEXT SHIFT,EUPHEMISM,FALSE ASSERTION,HYPERBOLE,OTHER,OXYMORON,RHETORICAL QUESTION
88,9408805770120683520,2035190000000000000,Alessandro,0,0,0,0,1,0,0,0
289,9408805770120683520,2035190000000000000,Cristina,0,0,0,0,1,0,0,0
490,9408805770120683520,2035190000000000000,Eliana,0,0,0,0,1,0,0,0
691,9408805770120683520,2035190000000000000,Michele,0,0,0,0,1,0,0,0
892,9408805770120683520,2035190000000000000,Michael,0,0,0,0,1,0,0,0
1093,9408805770120683520,2035190000000000000,Elisa,0,0,0,0,1,0,0,0


### su triple (insieme delle parti)

In [127]:
import itertools
import pandas as pd

annotators = list(final_cleaned.keys())

# Dimensioni gruppi (esempio: metà e metà)
group1_size = len(annotators) // 2
group2_size = len(annotators) - group1_size

# Tutte le combinazioni di annotatori
all_combinations = list(itertools.combinations(annotators, group1_size))
print(f"Numero totale combinazioni: {len(all_combinations)}")

# Istanza uniche (post_id, reply_id)
instances = full_df[['post_id', 'reply_id']].drop_duplicates()
rhetorical_figures = all_figures  # lista delle figure retoriche

# Calcolo P-index medio per ogni combinazione di gruppi
p_index_means = []

for i, group1 in enumerate(all_combinations, 1):
    group2 = [a for a in annotators if a not in group1]
    group_map = {a: 'group1' if a in group1 else 'group2' for a in annotators}
    
    p_index_per_instance = {}

    for _, row in instances.iterrows():
        pid, rid = row['post_id'], row['reply_id']
        instance_df = full_df[(full_df['post_id'] == pid) & (full_df['reply_id'] == rid)]

        p_indexes = []
        for fig in rhetorical_figures:
            df_subset = instance_df[['annotator', fig]].copy()
            if df_subset[fig].sum() + (df_subset[fig] == 0).sum() < 2:
                continue  # Troppo pochi annotatori
            p = compute_p_index_for_message_fixed(df_subset.assign(dummy_id=1), fig, group_map)
            p_indexes.append(p)

        if p_indexes:
            p_index_per_instance[(pid, rid)] = np.mean(p_indexes)
    
    # Media dei p-index per istanza in questa combinazione
    if p_index_per_instance:
        mean_p_index = np.mean(list(p_index_per_instance.values()))
        p_index_means.append(mean_p_index)

    if i % 10 == 0 or i == len(all_combinations):
        print(f"Calcolate {i}/{len(all_combinations)} combinazioni, ultima media P-index: {mean_p_index:.3f}")

# Statistiche finali
overall_mean_random = np.mean(p_index_means)
print(f"\nMedia del P-index su tutte le permutazioni: {overall_mean_random:.3f}")

Numero totale combinazioni: 20
Calcolate 10/20 combinazioni, ultima media P-index: 0.136
Calcolate 20/20 combinazioni, ultima media P-index: 0.131

Media del P-index su tutte le permutazioni: 0.132


### Delta

In [128]:
def calc_delta(real, random):
    return ((real - random) / random) * 100

print("\nDelta tra P-index reale e medio casuale:")
print(f"  Reale: {mean_p_index_real:.3f}")
print(f"  Casual: {overall_mean_random:.3f}")

print(calc_delta(mean_p_index_real, overall_mean_random))


Delta tra P-index reale e medio casuale:
  Reale: 0.124
  Casual: 0.132
-6.100450680667162
